In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import os
from functools import reduce
import warnings 
warnings.filterwarnings('ignore')

In [2]:
# 분류모델 import 
# Logistic Regression,SGD,KNN,SVM,Decision Tree, Random Forest, LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report,roc_auc_score,f1_score

lr = LogisticRegression()
knn = KNeighborsClassifier()
svm = SVC()
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier()
lgbm = LGBMClassifier()

In [3]:
df_coldwave = pd.read_csv('./preprocessed/df_coldwave.csv',encoding='cp949',index_col=0)
df_near = pd.read_csv('./preprocessed/preprocessed_주변국 기상데이터_199601to202108.csv',index_col=0)

In [4]:
df_coldwave.drop(['연도','날짜','지점'],axis=1,inplace=True)

In [5]:
# df_coldwave.reset_index(inplace=True)

In [6]:
# 중복된 날짜 제거 
df_coldwave = df_coldwave.drop_duplicates()

In [7]:
df_coldwave

,일시,재난일어난날
0,1973-01-03,1973-01-03
16,1973-01-04,1973-01-04
18,1973-01-06,1973-01-06
19,1973-01-11,1973-01-11
22,1973-01-12,1973-01-12
...,...,...
21950,2022-02-20,2022-02-20
21952,2022-02-21,2022-02-21
21960,2022-02-22,2022-02-22
21963,2022-02-23,2022-02-23


In [8]:
df_near

,도시명,일시,평균기온,최고기온,최저기온,습도,풍향,풍속,강수량,해면기압,이슬점 온도,군집
0,청진,1996-01-01,-5.528571,-0.1,-9.9,53.571429,330.0,0.571429,0.0,1018.685714,-13.657143,n2
1,청진,1996-01-02,-3.825000,1.3,-8.2,72.875000,80.0,0.500000,0.0,1014.037500,-8.025000,n2
2,청진,1996-01-03,-6.912500,-1.4,-11.7,48.375000,330.0,3.000000,0.0,1010.537500,-16.100000,n2
3,청진,1996-01-04,-9.475000,-3.4,-19.2,56.500000,320.0,0.250000,0.0,1020.800000,-16.625000,n2
4,청진,1996-01-05,-4.487500,0.5,-9.9,71.750000,290.0,0.375000,0.0,1021.800000,-8.837500,n2
...,...,...,...,...,...,...,...,...,...,...,...,...
9370,후쿠오카,2021-08-27,29.175000,32.8,25.6,70.495833,30.0,5.125000,0.0,1014.391667,23.162500,n5
9371,후쿠오카,2021-08-28,28.641667,32.3,25.1,72.362500,40.0,4.958333,0.0,1014.325000,23.000000,n5
9372,후쿠오카,2021-08-29,28.908333,32.4,25.5,72.687500,290.0,5.041667,0.0,1014.754167,23.375000,n5
9373,후쿠오카,2021-08-30,28.904167,32.5,25.8,75.795833,310.0,4.541667,0.0,1015.587500,24.112500,n5


In [9]:
# 주변국과 한파를 merge
cold_near = pd.merge(df_coldwave,df_near, how='right',on='일시')

In [10]:
cold_near

,일시,재난일어난날,도시명,평균기온,최고기온,최저기온,습도,풍향,풍속,강수량,해면기압,이슬점 온도,군집
0,1996-01-01,1996-01-01,청진,-5.528571,-0.1,-9.9,53.571429,330.0,0.571429,0.0,1018.685714,-13.657143,n2
1,1996-01-02,1996-01-02,청진,-3.825000,1.3,-8.2,72.875000,80.0,0.500000,0.0,1014.037500,-8.025000,n2
2,1996-01-03,NaN,청진,-6.912500,-1.4,-11.7,48.375000,330.0,3.000000,0.0,1010.537500,-16.100000,n2
3,1996-01-04,1996-01-04,청진,-9.475000,-3.4,-19.2,56.500000,320.0,0.250000,0.0,1020.800000,-16.625000,n2
4,1996-01-05,1996-01-05,청진,-4.487500,0.5,-9.9,71.750000,290.0,0.375000,0.0,1021.800000,-8.837500,n2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
271870,2021-08-27,NaN,후쿠오카,29.175000,32.8,25.6,70.495833,30.0,5.125000,0.0,1014.391667,23.162500,n5
271871,2021-08-28,NaN,후쿠오카,28.641667,32.3,25.1,72.362500,40.0,4.958333,0.0,1014.325000,23.000000,n5
271872,2021-08-29,NaN,후쿠오카,28.908333,32.4,25.5,72.687500,290.0,5.041667,0.0,1014.754167,23.375000,n5
271873,2021-08-30,NaN,후쿠오카,28.904167,32.5,25.8,75.795833,310.0,4.541667,0.0,1015.587500,24.112500,n5


In [11]:
# 재난일어난날 컬럼을 바이너리로 만들기 
cold_near['재난일어난날'] = cold_near['재난일어난날'].fillna(0)

In [12]:
cold_near['재난일어난날'] =cold_near['재난일어난날'].apply(lambda x: x if str(x) == '0' else '1')

In [13]:
# 쓸모없는 column 제거
cold_near.drop('일시',axis=1,inplace=True)

In [14]:
# 전부 뉴메릭으로 만들기 
cold_near['재난일어난날']=cold_near['재난일어난날'].astype(int)

In [15]:
# 위치를 원핫인코딩하기
cold_near = pd.get_dummies(cold_near)

In [16]:
# 재난일어난날 2주뒤로 미뤄서 예측값 찾기 
cold_near['재난일어난날'] = cold_near['재난일어난날'].shift(periods=14, axis=0)

In [17]:
# 뒤로미룬만큼 생긴 nan값 제거 
cold_near.dropna(inplace=True)

In [18]:
cold_near.head()

,재난일어난날,평균기온,최고기온,최저기온,습도,풍향,풍속,강수량,해면기압,이슬점 온도,...,도시명_해주,도시명_혜산,도시명_후쿠오카,군집_n1,군집_n2,군집_n3,군집_n4,군집_n5,군집_n6,군집_n7
14,1.0,-2.514286,-1.0,-4.8,70.000000,30.0,0.285714,0.0,1019.600000,-7.357143,...,0,0,0,0,1,0,0,0,0,0
15,1.0,-6.612500,-3.7,-9.2,51.500000,40.0,0.375000,0.0,1020.000000,-15.150000,...,0,0,0,0,1,0,0,0,0,0
16,0.0,-6.471429,-3.0,-9.2,67.857143,310.0,0.571429,0.0,1022.457143,-11.457143,...,0,0,0,0,1,0,0,0,0,0
17,1.0,-5.228571,-3.4,-8.0,60.285714,40.0,1.375000,0.0,1025.050000,-11.862500,...,0,0,0,0,1,0,0,0,0,0
18,1.0,-5.362500,-0.6,-9.7,71.125000,90.0,0.250000,0.0,1018.787500,-9.850000,...,0,0,0,0,1,0,0,0,0,0


In [19]:
cold_near['재난일어난날'] = cold_near['재난일어난날'].astype(int)

In [20]:
cold_near

,재난일어난날,평균기온,최고기온,최저기온,습도,풍향,풍속,강수량,해면기압,이슬점 온도,...,도시명_해주,도시명_혜산,도시명_후쿠오카,군집_n1,군집_n2,군집_n3,군집_n4,군집_n5,군집_n6,군집_n7
14,1,-2.514286,-1.0,-4.8,70.000000,30.0,0.285714,0.0,1019.600000,-7.357143,...,0,0,0,0,1,0,0,0,0,0
15,1,-6.612500,-3.7,-9.2,51.500000,40.0,0.375000,0.0,1020.000000,-15.150000,...,0,0,0,0,1,0,0,0,0,0
16,0,-6.471429,-3.0,-9.2,67.857143,310.0,0.571429,0.0,1022.457143,-11.457143,...,0,0,0,0,1,0,0,0,0,0
17,1,-5.228571,-3.4,-8.0,60.285714,40.0,1.375000,0.0,1025.050000,-11.862500,...,0,0,0,0,1,0,0,0,0,0
18,1,-5.362500,-0.6,-9.7,71.125000,90.0,0.250000,0.0,1018.787500,-9.850000,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271870,0,29.175000,32.8,25.6,70.495833,30.0,5.125000,0.0,1014.391667,23.162500,...,0,0,1,0,0,0,0,1,0,0
271871,0,28.641667,32.3,25.1,72.362500,40.0,4.958333,0.0,1014.325000,23.000000,...,0,0,1,0,0,0,0,1,0,0
271872,0,28.908333,32.4,25.5,72.687500,290.0,5.041667,0.0,1014.754167,23.375000,...,0,0,1,0,0,0,0,1,0,0
271873,0,28.904167,32.5,25.8,75.795833,310.0,4.541667,0.0,1015.587500,24.112500,...,0,0,1,0,0,0,0,1,0,0


In [21]:
# 한파주변국 데이터 전처리 끝 모델돌리기 

In [22]:
y = cold_near.재난일어난날
x = cold_near.drop('재난일어난날',axis=1)

In [23]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [24]:
models = [lr,dtc,rfc,lgbm]
# result = []
for i in models:
    i.fit(x_train,y_train)
    pred = i.predict(x_test)
    proba = i.predict_proba(x_test)[:,1]
    
    print(i.__class__.__name__)
    print(classification_report(y_test,pred))
    print('roc_auc_score :',roc_auc_score(y_test,proba))
    print('-'*50)

LogisticRegression
              precision    recall  f1-score   support

           0       0.89      0.98      0.93     47982
           1       0.43      0.13      0.20      6391

    accuracy                           0.88     54373
   macro avg       0.66      0.55      0.56     54373
weighted avg       0.84      0.88      0.85     54373

roc_auc_score : 0.884098220450256
--------------------------------------------------
DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.92      0.91      0.92     47982
           1       0.37      0.39      0.38      6391

    accuracy                           0.85     54373
   macro avg       0.65      0.65      0.65     54373
weighted avg       0.85      0.85      0.85     54373

roc_auc_score : 0.6509393833932704
--------------------------------------------------
RandomForestClassifier
              precision    recall  f1-score   support

           0       0.90      0.97      0.94     47982
 

In [25]:
#f1 score가 너무 낮게나왔으니 임밸런스 데이터를 제거하고 다시 

In [26]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [27]:
scaler.fit(x_train)

StandardScaler()

In [28]:
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [34]:
y_train.value_counts()

0    191689
1     25799
Name: 재난일어난날, dtype: int64

In [30]:
import imblearn
from imblearn.over_sampling import SMOTE

In [31]:
smote = SMOTE(random_state=0)
x_train_sm,y_train_sm = smote.fit_resample(x_train_scaled,y_train)

In [32]:
y_train_sm.value_counts()

0    191689
1    191689
Name: 재난일어난날, dtype: int64

In [33]:
models = [lr,dtc,rfc,lgbm]
# result = []
for i in models:
    i.fit(x_train_sm,y_train_sm)
    pred = i.predict(x_test_scaled)
    proba = i.predict_proba(x_test_scaled)[:,1]
    
    print(i.__class__.__name__)
    print(classification_report(y_test,pred))
    print('roc_auc_score :',roc_auc_score(y_test,proba))
    print('-'*50)
    

LogisticRegression
              precision    recall  f1-score   support

           0       0.98      0.79      0.87     47982
           1       0.36      0.89      0.51      6391

    accuracy                           0.80     54373
   macro avg       0.67      0.84      0.69     54373
weighted avg       0.91      0.80      0.83     54373

roc_auc_score : 0.8979514536696371
--------------------------------------------------
DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.93      0.88      0.90     47982
           1       0.35      0.46      0.40      6391

    accuracy                           0.83     54373
   macro avg       0.64      0.67      0.65     54373
weighted avg       0.86      0.83      0.84     54373

roc_auc_score : 0.6742996925625653
--------------------------------------------------
RandomForestClassifier
              precision    recall  f1-score   support

           0       0.96      0.86      0.91     47982
